<a href="https://colab.research.google.com/github/lucianoecp/DE/blob/main/pyomo_pibic_caso1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-08-28 07:12:04--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’

Miniconda3-py37_4.8 100%[===================>]  81.12M   145MB/s    in 0.6s    

2021-08-28 07:12:05 (145 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...


In [ ]:
! conda install -c rdkit rdkit -y

In [ ]:
!conda update conda -y
!conda update anaconda -y

In [ ]:
!conda install -c conda-forge pyomo -y
!conda install -c conda-forge pyomo.extras -y

In [ ]:
!conda install -c conda-forge glpk -y
!conda install -c conda-forge coincbc -y
!conda install -c conda-forge ipopt -y

In [ ]:
# -*- coding: utf-8 -*-


from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

import pyomo.environ as pyo, numpy as np, pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Entrada de dados:
#
## Geradores
arquivo = 'input4.xlsx'
ute = pd.read_excel(arquivo, sheet_name='UTE')
demanda = pd.read_excel(arquivo, sheet_name='demanda')

### termoelétricas:
cos = ute.Custo.values
a = ute.A.values*cos
b = ute.B.values*cos
c = ute.C.values*cos
pmin = ute.Pmin.values
pmax = ute.Pmax.values
ur = ute.UR.values
dr = ute.DR.values

## Demanda
Pd = demanda.dem.values
t = demanda.t.to_list()

Nd = len(Pd)
Ng = len(ute)

model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), range(Nd), bounds=(0,None))
Pg = model.Pg

model.deficit = pyo.Var(range(Nd), bounds=(0,None))
deficit = model.deficit

def fg(Pg,a,b,c,t):
    return sum([a[g] + b[g]*Pg[g,t] + c[g]*Pg[g,t]*Pg[g,t] for g in range(Ng)])

def fun_custo_total(Pg,a,b,c,Ph,ah,bh,ch,Pw,disp,Wmax,t):
    Fg = fg(Pg,a,b,c,t)
    Fh = fh(Ph,ah,bh,ch,t)
    Fw = fw(Pw,disp,Wmax,t)
    return Fg+Fh+Fw

# Objetivo
model.obj = pyo.Objective(expr= sum(fg(Pg,a,b,c,t) + deficit[t]*99999 for t in range(Nd)), sense=minimize)

# balanco de potencia:
model.balanco = ConstraintList()
for t in range(Nd):
    sum_Pg = sum(Pg[g,t] for g in range(Ng))
    model.balanco.add(expr= (sum_Pg + deficit[t]) == Pd[t])

# limites geração termo:
model.limger = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        model.limger.add(inequality(pmin[g], Pg[g,t], pmax[g]))

# limites de rampas
model.ramp_up = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        if(t==0):
            Constraint.Skip
        else:
            model.ramp_up.add(expr= Pg[g,t] - Pg[g,t-1] <= ur[g])

model.ramp_down = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        if(t==0):
            Constraint.Skip
        else:
            model.ramp_down.add(expr= Pg[g,t-1] - Pg[g,t] <= dr[g])

#opt = SolverFactory('couenne', executable='C:\\couenne\\bin\\couenne.exe')
opt = SolverFactory('mindtpy')
opt.solve(model, mip_solver='glpk', nlp_solver='ipopt')
#opt.solve(model)

model.pprint()

# 
f = open('caso1.csv', 'w')

f.write("t,Pd,")
for g in range(Ng):
    f.write(f"G[{g+1}]"+",")
f.write("Gerado,Corte"+"\n")


for t in range(Nd):
    f.write(str(t)+","+str(Pd[t])+",")
    for g in range(Ng):
        f.write(str(pyo.value(Pg[g,t]))+",")
    f.write(str(sum(pyo.value(Pg[:,t])))+ "," + str(int(pyo.value(deficit[t])))+"\n")



In [ ]:
print(ute)
print("\n")
print(Pd)

In [ ]:
data = pd.read_csv('caso1.csv')
data.head()
print(data)